#### Distance Algorithm Comparison

In [14]:
import os
import time
import pandas as pd
import osmnx as ox
import networkx as nx
from tqdm import tqdm
import numpy as np
import math
from network_cache import cache_greater_vancouver_network 

##### Configuration

In [15]:
# Coordinates of the start location (Northeastern University - Vancouver)
start_coords = (49.28069261104817, -123.11572527879807)

# GraphML file and housing CSV
graph_file = "greater_vancouver.graphml"
csv_file = "house_with_coordinates.csv"

In [16]:
# Load road network graph and housing data

graph_path = os.path.abspath(graph_file)
# If the graph does not exist, generate it using OSMnx
if not os.path.exists(graph_path):
    print("Graph file not found. Generating now...")
    cache_greater_vancouver_network(output_path=graph_path)
df = pd.read_csv(csv_file)

if not os.path.exists(graph_path):
    raise FileNotFoundError("GraphML file not found. Run network_cache.py first.")

if not {'Latitude', 'Longitude'}.issubset(df.columns):
    raise ValueError("CSV must contain 'Latitude' and 'Longitude' columns.")

print("Loading graph...")
G = ox.load_graphml(graph_path)

# Get the nearest network node to the start point
start_node = ox.distance.nearest_nodes(G, X=start_coords[1], Y=start_coords[0])
print("Start node ID:", start_node)


Loading graph...
Start node ID: 11106039371


##### Dijkstra Distance Calculation

In [17]:
def compute_dijkstra_distances():
    distances = []
    start_time = time.time()
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Dijkstra"):
        try:
            house_coords = (row['Latitude'], row['Longitude'])
            house_node = ox.distance.nearest_nodes(G, X=house_coords[1], Y=house_coords[0])
            length = nx.shortest_path_length(G, start_node, house_node, weight='length')
        except Exception:
            length = None
        distances.append(length)
    
    elapsed = time.time() - start_time
    return distances, elapsed

##### A* Distance Calculation

In [18]:
# Great-circle distance in meters (admissible lower bound of road distance)
def haversine_heuristic(u, v):
    # u, v are node IDs in G
    lat1, lon1 = G.nodes[u]['y'], G.nodes[u]['x']
    lat2, lon2 = G.nodes[v]['y'], G.nodes[v]['x']

    R = 6371000.0  # Earth radius in meters
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)

    a = math.sin(d_phi / 2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2.0)**2
    c = 2.0 * math.atan2(math.sqrt(a), math.sqrt(1.0 - a))
    return R * c  # meters

def compute_astar_distances():
    distances = []
    start_time = time.time()

    for _, row in tqdm(df.iterrows(), total=len(df), desc="A*"):
        try:
            house_coords = (row['Latitude'], row['Longitude'])
            house_node = ox.distance.nearest_nodes(G, X=house_coords[1], Y=house_coords[0])

            length = nx.astar_path_length(
                G,
                start_node,
                house_node,
                weight='length',               # edge lengths in meters
                heuristic=haversine_heuristic  # admissible heuristic
            )
        except Exception:
            length = None

        distances.append(length)

    elapsed = time.time() - start_time
    return distances, elapsed


##### Single-source Dijkstra: with loop 

In [27]:
def compute_single_source_dijkstra_with_loop():
    """
    Single-source Dijkstra with per-house nearest node lookup
    """
    distances = []
    start_time = time.time()

    # Run Dijkstra from the start node once
    try:
        path_lengths = nx.single_source_dijkstra_path_length(G, start_node, weight='length')
    except Exception:
        path_lengths = {}

    # Per-row nearest_nodes lookup
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Dijkstra + Loop"):
        try:
            house_node = ox.distance.nearest_nodes(G, X=row['Longitude'], Y=row['Latitude'])
            dist = path_lengths.get(house_node, None)
        except Exception:
            dist = None
        distances.append(dist)

    elapsed = time.time() - start_time
    return distances, elapsed


##### Single-source Dijkstra: with per-house nearest node lookup

In [20]:
def compute_fast_single_source_dijkstra_distances():
    start_time = time.time()

    # Run single-source Dijkstra from the start node
    path_lengths = nx.single_source_dijkstra_path_length(G, start_node, weight='length')

    # Vectorized lookup of house nodes
    house_nodes = ox.distance.nearest_nodes(
        G, X=df['Longitude'].values, Y=df['Latitude'].values, return_dist=False
    )

    # Lookup distance for each house node from precomputed dict
    distances = [path_lengths.get(node, None) for node in house_nodes]

    elapsed = time.time() - start_time
    return distances, elapsed


##### Run and Compare

In [21]:
print("Running Dijkstra...")
dijkstra_distances, dijkstra_time = compute_dijkstra_distances()

print("\nRunning A*...")
astar_distances, astar_time = compute_astar_distances()



Running Dijkstra...


Dijkstra: 100%|██████████| 1302/1302 [01:31<00:00, 14.19it/s]



Running A*...


A*: 100%|██████████| 1302/1302 [01:21<00:00, 16.04it/s]


In [22]:
print("\nRunning Single-Source Dijkstra (with loop)...")
single_source_loop_distances, single_source_loop_time = compute_single_source_dijkstra_with_loop()


Running Single-Source Dijkstra (with loop)...


Dijkstra + Loop: 100%|██████████| 1302/1302 [01:15<00:00, 17.21it/s]


In [23]:
print("\nRunning Fast Dijkstra (with per-house nearest node lookup)...")
fast_dijkstra_distances, fast_dijkstra_time = compute_fast_single_source_dijkstra_distances()


Running Fast Dijkstra (with per-house nearest node lookup)...


##### Results Summary

In [24]:
print("\nElapsed Time:")
print(f"Dijkstra                : {dijkstra_time:.2f} seconds")
print(f"A*                      : {astar_time:.2f} seconds")
print(f"Single-Source + Loop    : {single_source_loop_time:.2f} seconds")
print(f"Fast Dijkstra (Vector)  : {fast_dijkstra_time:.2f} seconds")



Elapsed Time:
Dijkstra                : 91.75 seconds
A*                      : 81.18 seconds
Single-Source + Loop    : 75.78 seconds
Fast Dijkstra (Vector)  : 0.18 seconds


##### Compare the results

In [25]:
# Convert all to arrays with float64 (None → np.nan)
dijkstra_arr = np.array(dijkstra_distances, dtype='float64')
astar_arr = np.array(astar_distances, dtype='float64')
loop_arr = np.array(single_source_loop_distances, dtype='float64')
fast_arr = np.array(fast_dijkstra_distances, dtype='float64')

# Set tolerance for floating-point comparison
rtol = 1e-5
atol = 1e-8

# Helper to count mismatches
def count_mismatch(arr1, arr2):
    return np.sum(~np.isclose(arr1, arr2, rtol=rtol, atol=atol) & ~np.isnan(arr1) & ~np.isnan(arr2))

# Count mismatches vs Dijkstra
mismatch_astar = count_mismatch(dijkstra_arr, astar_arr)
mismatch_loop = count_mismatch(dijkstra_arr, loop_arr)
mismatch_fast = count_mismatch(dijkstra_arr, fast_arr)

print("\nDistance Value Comparison (vs Original Dijkstra):")
print(f"A*                      : {mismatch_astar} mismatches")
print(f"Single-Source + Loop    : {mismatch_loop} mismatches")
print(f"Fast Dijkstra (Vector)  : {mismatch_fast} mismatches")

if mismatch_astar == 0 and mismatch_loop == 0 and mismatch_fast == 0:
    print("All algorithms match Original Dijkstra within tolerance.")
else:
    print("Some algorithms produced different results.")

# Optional: Show example mismatches
if mismatch_fast > 0:
    mismatch_df = df.copy()
    mismatch_df["Dijkstra"] = dijkstra_arr
    mismatch_df["Fast_Dijkstra"] = fast_arr
    mismatch_df = mismatch_df[
        ~np.isclose(dijkstra_arr, fast_arr, rtol=rtol, atol=atol) &
        ~mismatch_df["Dijkstra"].isna() &
        ~mismatch_df["Fast_Dijkstra"].isna()
    ]
    print("\nExamples of mismatches between Dijkstra and Fast Dijkstra:")
    display(mismatch_df.head(5)[["Address", "Dijkstra", "Fast_Dijkstra"]])


Distance Value Comparison (vs Original Dijkstra):
A*                      : 0 mismatches
Single-Source + Loop    : 0 mismatches
Fast Dijkstra (Vector)  : 0 mismatches
All algorithms match Original Dijkstra within tolerance.


In [26]:
# Create a copy of the original dataframe
result_df = df.copy()

# Add all four distance columns
result_df["Dijkstra_Distance"] = dijkstra_distances
result_df["Astar_Distance"] = astar_distances
result_df["Loop_Dijkstra_Distance"] = single_source_loop_distances
result_df["Fast_Dijkstra_Distance"] = fast_dijkstra_distances

# Save to CSV
result_df.to_csv("distance_comparison_results.csv", index=False)
print("Saved to 'distance_comparison_results.csv'")

# Show preview
result_df.head()


Saved to 'distance_comparison_results.csv'


,Number,Address,List Date,Price,Days on market,Total floor area,Year Built,Age,Lot Size,Latitude,Longitude,Dijkstra_Distance,Astar_Distance,Loop_Dijkstra_Distance,Fast_Dijkstra_Distance
0,1,3178 GRAVELEY STREET,5/8/2020,1500000,18,2447,1946,74,5674.00,49.270200,-123.037009,6472.196776,6472.196776,6472.196776,6472.196776
1,2,1438 E 28TH AVENUE,1/22/2020,1300000,7,2146,1982,38,3631.98,49.245161,-123.074991,5827.364178,5827.364178,5827.364178,5827.364178
2,3,2831 W 49TH AVENUE,6/18/2019,2650000,1,3108,1929,90,9111.00,49.227603,-123.168387,8778.309479,8778.309479,8778.309479,8778.309479
3,4,2645 TRIUMPH STREET,6/18/2019,1385000,28,2602,1922,97,4022.70,49.284068,-123.050868,4942.984031,4942.984031,4942.984031,4942.984031
4,5,741-743 E 10TH AVENUE,11/28/2019,1590000,17,1843,1970,49,4026.00,49.261739,-123.088100,3754.071589,3754.071589,3754.071589,3754.071589
